In [1]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df = pd.read_csv("roles_based_on_skills_train (1).csv")
test_df = pd.read_csv("roles_based_on_skills_test (1).csv")

In [3]:
# Check column names in train and test datasets
print("Train columns:", train_df.columns)
print("Test columns:", test_df.columns)

Train columns: Index(['Role', 'text', 'label', '__index_level_0__'], dtype='object')
Test columns: Index(['Role', 'text', 'label', '__index_level_0__'], dtype='object')


In [4]:
label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["Role"])
test_df["label"] = label_encoder.transform(test_df["Role"])

In [5]:
train_df = train_df.rename(columns={"skills": "text"})
test_df = test_df.rename(columns={"skills": "text"})


In [6]:
# 5. Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [7]:
# 6. Load Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)


In [9]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3660 [00:00<?, ? examples/s]

Map:   0%|          | 0/916 [00:00<?, ? examples/s]

In [10]:
# 7. Format datasets for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [11]:
# 8. Load DistilBERT Model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_encoder.classes_)
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
!pip install --upgrade transformers


In [13]:
from transformers import TrainingArguments


In [14]:
# 9. Training Arguments
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=8, per_device_eval_batch_size=8,  logging_dir="./logs", num_train_epochs=3)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [16]:
!pip install --upgrade datasets


In [17]:
# 11. Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayeshahussain005 (ayeshahussain005-gift-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.642100
1000,0.229200


TrainOutput(global_step=1374, training_loss=0.3522006510994146, metrics={'train_runtime': 639.0814, 'train_samples_per_second': 17.181, 'train_steps_per_second': 2.15, 'total_flos': 363674886912000.0, 'train_loss': 0.3522006510994146, 'epoch': 3.0})

In [18]:
model.save_pretrained("./distilbert_recommender")
tokenizer.save_pretrained("./distilbert_recommender")


('./distilbert_recommender/tokenizer_config.json',
 './distilbert_recommender/special_tokens_map.json',
 './distilbert_recommender/vocab.txt',
 './distilbert_recommender/added_tokens.json',
 './distilbert_recommender/tokenizer.json')

In [19]:
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [20]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

# Load model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("./distilbert_recommender")
tokenizer = DistilBertTokenizerFast.from_pretrained("./distilbert_recommender")

# Load label encoder
import pickle
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)


In [21]:

# Prediction function
def predict_user_input(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([prediction])[0]



In [24]:
# Example usage
user_input = "SEO, Content Writing, Google Analytics, Marketing Strategy"
predicted_role = predict_user_input(user_input)
print("Recommended Role:", predicted_role)


Recommended Role: Cyber Security


In [25]:
# Example usage
user_input = "Linux, Docker, Kubernetes, AWS"
predicted_role = predict_user_input(user_input)
print("Recommended Role:", predicted_role)


Recommended Role: Machine Learning Engineer


In [27]:
# Example usage
user_input = "SQL, Data Analysis, Business Intelligence, Power BI"
predicted_role = predict_user_input(user_input)
print("Recommended Role:", predicted_role)


Recommended Role: Data Engineer
